<a href="https://colab.research.google.com/github/MarVinikoi/DL_NafldReserch/blob/main/ImageClassif_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install keras-tcn
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
%cd "/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/images/"

/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/images


In [14]:
from tcn import TCN, tcn_full_summary
from tensorflow.python.keras.models import Sequential
import cv2
import tensorflow as tf
import numpy as np
import glob, os
from sklearn.model_selection import train_test_split
import math
import random
from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#Variáveis Globais

path_doentes = "doentes"
path_saudaveis = "nao_doentes"

In [16]:

def dataAugmentation(on = False):
  if on:
    for paciente in os.listdir(path_doentes):
      path_paciente = os.path.join(path_doentes, paciente)
      paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
      for i, img_path in enumerate(paths_imgs):
        image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        vflip_image_name = nameFormatter(img_path, "Vflip.jpg") 
        hflip_image_name = nameFormatter(img_path, "Hflip.jpg")
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)


    for paciente in os.listdir(path_saudaveis):
      path_paciente = os.path.join(path_saudaveis, paciente)
      paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
      for i, img_path in enumerate(paths_imgs):
        image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        vflip_image_name = nameFormatter(img_path, "Vflip.jpg") 
        hflip_image_name = nameFormatter(img_path, "Hflip.jpg")
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)

def kerasDataAug(on = False):
  if on:
    imgGen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)

    for paciente in os.listdir(path_doentes):
      path_paciente = os.path.join(path_doentes, paciente)
      #for i, img in enumerate(os.listdir(os.path.join(path_doentes, paciente))):
      paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
      for i, img_path in enumerate(paths_imgs):
        pass


def nameFormatter(filepath = "", textToConcat = ""):
  '''
  @param path_opt - 0 para o path de pacientes doentes e qualquer outro valor para saudáveis
  @param string1 - nome do arquivo que foi alterado nas operações de Data Aug
  @param string2 - string que identifica a operação realizada na imagem
  @param string3 - paciente atual cuja imagem modificada pertence
  '''

  dot_index = filepath.find(".")
  filepath = filepath[:dot_index]
  filename = filepath + textToConcat
  return filename


In [52]:
def main():
  dataAugmentation(False) #Só executa com o parametro setado em True

  loadData()
  
  

if __name__ == "__main__":
  main()

doentes/doente_7
doentes/doente_7/IR_41473.jpg
doentes/doente_7/IR_41473Hflip.jpg
doentes/doente_7/IR_41473Vflip.jpg
doentes/doente_7/IR_41475.jpg
doentes/doente_7/IR_41475Hflip.jpg
doentes/doente_7/IR_41475Vflip.jpg
doentes/doente_7/IR_41477.jpg
doentes/doente_7/IR_41477Hflip.jpg
doentes/doente_7/IR_41477Vflip.jpg
doentes/doente_7/IR_41479.jpg
doentes/doente_7/IR_41479Hflip.jpg
doentes/doente_7/IR_41479Vflip.jpg
doentes/doente_7/IR_41481.jpg
doentes/doente_7/IR_41481Hflip.jpg
doentes/doente_7/IR_41481Vflip.jpg
doentes/doente_1
doentes/doente_1/IR_43059.jpg
doentes/doente_1/IR_43059Hflip.jpg
doentes/doente_1/IR_43059Vflip.jpg
doentes/doente_1/IR_43061.jpg
doentes/doente_1/IR_43061Hflip.jpg
doentes/doente_1/IR_43061Vflip.jpg
doentes/doente_1/IR_43063.jpg
doentes/doente_1/IR_43063Hflip.jpg
doentes/doente_1/IR_43063Vflip.jpg
doentes/doente_1/IR_43065.jpg
doentes/doente_1/IR_43065Hflip.jpg
doentes/doente_1/IR_43065Vflip.jpg
doentes/doente_1/IR_43067.jpg
doentes/doente_1/IR_43067Hflip.jpg
d

In [51]:


def loadData():


  doentes = []
  saudaveis = []
  
  treino = []
  teste = []
  val = []

  labels_treino = []
  labels_teste = []
  labels_val = []

  doentes_size = 91
  saudaveis_size = 132
  all_size = 223

  vetor_imgs = []
  vetor_final_imgs_doentes = []
  vetor_final_imgs_saudaveis = []


  #Carregando os vetores de imagens e os transformando em séries temporais
  #Vetor de Doentes
  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    #for i, img in enumerate(os.listdir(os.path.join(path_doentes, paciente))):
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(i < 15):
        image = cv2.imread(img_path)
        print(img_path)
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)

      #image_array = np.concatenate((r_img, g_img, b_img))
      vetor_imgs.append(r_img)
    
    vetor_final_imgs_doentes.append((vetor_imgs[0], vetor_imgs[3], vetor_imgs[6], vetor_imgs[9],
                                     vetor_imgs[12]))
    vetor_imgs.clear()
  vetor_final_imgs_doentes = np.array(vetor_final_imgs_doentes)  
  print(vetor_final_imgs_doentes.shape)
  


  #Vetor de Saudáveis
  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(i < 15):
      
        image = cv2.imread(img_path)
        print(img_path)
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)
        

        #image_array = np.concatenate((r_img,g_img, b_img))
            
        vetor_imgs.append(r_img)
    vetor_final_imgs_saudaveis.append((vetor_imgs[0], vetor_imgs[3], vetor_imgs[6], vetor_imgs[9],
                                     vetor_imgs[12]))
    vetor_imgs.clear()

  vetor_final_imgs_saudaveis = np.array(vetor_final_imgs_saudaveis)  
  print(vetor_final_imgs_saudaveis.shape)

  n_imgs_treino_doe = round((len(vetor_final_imgs_doentes)*0.65))
  n_imgs_treino_sau = round((len(vetor_final_imgs_saudaveis)*0.65))
  print(n_imgs_treino_doe)
  print(n_imgs_treino_sau)

  n_imgs_test_doe = n_imgs_treino_doe + round((len(vetor_final_imgs_doentes)*0.2))
  n_imgs_test_sau = n_imgs_treino_sau + round((len(vetor_final_imgs_saudaveis)*0.2))
  print(n_imgs_test_doe)
  print(n_imgs_test_sau)

  #n_imgs_val_doe = n_imgs_test_doe + math.floor((len(vetor_final_imgs_doentes)*0.15))
  #n_imgs_val_sau = n_imgs_test_sau + math.floor((len(vetor_final_imgs_saudaveis)*0.15))

  #Utilizando cerca de 65% das imagens para treino
  treino.extend(vetor_final_imgs_doentes[:n_imgs_treino_doe])
  treino.extend(vetor_final_imgs_saudaveis[:n_imgs_treino_sau])

  #Utilizando cerca de 20% das imagens para teste
  teste.extend(vetor_final_imgs_doentes[n_imgs_treino_doe:n_imgs_test_doe])
  teste.extend(vetor_final_imgs_saudaveis[n_imgs_treino_sau:n_imgs_test_sau])

  #Utilizando cerca de 15% das imagens para validação
  val.extend(vetor_final_imgs_doentes[n_imgs_test_doe:])
  val.extend(vetor_final_imgs_saudaveis[n_imgs_test_sau:])

  treino = np.array(treino)
  teste = np.array(teste)
  val = np.array(val)

  #LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
  for i in range(treino.shape[0]):
    if i < n_imgs_treino_doe:
      labels_treino.append(0)
    else:
      labels_treino.append(1)

  for i in range(teste.shape[0]):
    if i < (n_imgs_test_doe - n_imgs_treino_doe):
      labels_teste.append(0)
    else:
      labels_teste.append(1)

  for i in range(val.shape[0]):
    if i < (len(vetor_final_imgs_doentes) - n_imgs_test_doe):
      labels_val.append(0)
    else:
      labels_val.append(1)

  random.seed(2837465)

  random.shuffle(treino)
  random.shuffle(teste)
  random.shuffle(val)

  random.shuffle(labels_treino)
  random.shuffle(labels_teste)
  random.shuffle(labels_val)


  print(f"Labels de Treino: {labels_treino}")
  print(f"Labels de Teste: {labels_teste}")
  print(f"Labels de Validação: {labels_val}")
  
  labels_treino = np.array(labels_treino)
  labels_teste = np.array(labels_teste)
  labels_val = np.array(labels_val)


  print(treino.shape)
  print(teste.shape)
  print(val.shape)
  
  classifier(treino, teste, val, labels_treino, labels_teste, labels_val)


def classifier(train_set, test_set, val_set, train_label_set, test_label_set, val_label_set):

  model = Sequential([
      TCN(nb_filters = 64, dropout_rate=0.2, kernel_initializer="glorot_uniform",use_skip_connections=True),
      tf.keras.layers.Dense(units=1,activation="sigmoid")
  ])

  model.compile(optimizer="adam", loss = tf.losses.BinaryCrossentropy(), metrics = ["accuracy"])
  print("\n\n")
  
  hist = model.fit(train_set, train_label_set, validation_data=(val_set, val_label_set), epochs=60)

  model.summary()

  yhat = model.predict(test_set)
  print(f"Prediction Shape: {yhat}")
